In [1]:
import numpy as np

In [5]:
# Lire les données depuis le fichier
with open("german.data", "r") as file:
    data = file.readlines()

structured_data = []

# Parse each line and append to the list
for line in data:
    values = line.strip().split()
    structured_data.append(tuple(values))

# Create a NumPy array from the list
data = np.array(structured_data)




In [7]:
data.shape

(1000, 21)

In [8]:
unique_data = np.unique(data, axis=0)

In [9]:
unique_data.shape

(1000, 21)